In [14]:
import numpy as np
from numpy.fft import fftshift
 

def gf2_gaussian_elimination(A, b):
    """
    Solves Ax = b for x over GF(2) using Gaussian elimination.
    Returns x and a flag indicating if the solution is valid.
    """
    A, b = np.mod(A, 2), np.mod(b, 2)  # Ensure GF(2)
    m, n = A.shape
    Ab = np.hstack([A, b.reshape(-1, 1)])
    print("Ab", Ab)
    
    for col in range(n):
        for i in range(col, m):
            if Ab[i, col] == 1:
                if i != col:
                    Ab[[col, i]] = Ab[[i, col]]
                break
            print("Ab", Ab)
        else:
            continue
        
        for i in range(col + 1, m):
            if Ab[i, col] == 1:
                Ab[i] = np.mod(Ab[i] + Ab[col], 2)
            print("Ab", Ab)
    
    x = np.zeros(n, dtype=int)
    for i in range(n - 1, -1, -1):
        if Ab[i, i] == 0 and Ab[i, -1] == 1:
            print("Ab", Ab)
            return None, False  # Inconsistent system
        if Ab[i, i] == 1:
            print("Ab", Ab)
            x[i] = np.mod(Ab[i, n] + np.dot(Ab[i, i+1:n], x[i+1:n]), 2)
            return x, True

def Z_h(h, n):
    return np.mod(np.eye(2 * n) + np.mod(np.outer(fftshift(h), h), 2),2)
    
    

def symp_inn_pdt(X, Y):
    # Compute the symplectic inner product, assuming X and Y are in GF(2)
    return np.mod(np.sum(X * fftshift(Y, axes=1), axis=1), 2)

# def symp_mat_transvecs(h):
#     F = np.mod(np.dot(F, Z_h(h_i, n)), 2)

def find_symp_mat_transvecs(X, Y):
    m, cols = X.shape
    n = cols // 2
    F = np.eye(2 * n)
    Transvecs = []
    
    for i in range(m):
        x_i, y_i = X[i, :], Y[i, :]
        x_it = np.mod(np.dot(x_i, F), 2)
        if np.all(x_i == y_i):
            continue
        if symp_inn_pdt(x_i.reshape(1, -1), y_i.reshape(1, -1)) == 1:
            #print(symp_inn_pdt(x_it.reshape(1, -1), y_i.reshape(1, -1)))
            h_i = np.mod(x_it + y_i, 2)
            #print(h_i)
            F = np.mod(np.matmul(F, Z_h(h_i, n)), 2)
            #print(x_it)
            #print(np.mod((x_it+y_i),2))
            
            #print(Z_h(h_i, n))
            Transvecs.append((Z_h(h_i, n), h_i))
        else:
            w_i = find_w(x_it, y_i, Y[:i, :], n)

            print(w_i)
            h_i1 = np.mod(w_i + y_i, 2)

            print(h_i1)
            h_i2 = np.mod(x_it + w_i, 2)
            print(h_i2)
            F = np.mod(np.dot(np.dot(F, Z_h(h_i1, n)), Z_h(h_i2, n)), 2)
            Transvecs.append((Z_h(h_i1, n), h_i1))
            Transvecs.append((Z_h(h_i2, n), h_i2))
    return F, Transvecs

def find_w(x, y, Ys, n):
    A = fftshift(np.vstack([x, y] + [Ys[j, :] for j in range(Ys.shape[0])]), axes=1)
    print("a",A)
    b = np.array([1, 1] + [symp_inn_pdt(Ys[j, :].reshape(1, -1), y.reshape(1, -1))[0] for j in range(Ys.shape[0])])
    print("b",b)
    w, valid = gf2_gaussian_elimination(A, b)
    if not valid:
        raise ValueError("No valid solution found for w.")
    return w


In [15]:
# Define matrices X and Y
X = np.array([[1, 0,0,0], [0, 1,0,0],[0,0,1,0],[0,0,0,1]])
Y = np.array([[1,1,0,0], [0, 1,0,0],[0,0,1,0],[0,0,1,1]])


# Perform the symplectic matrix transformation
F, Transvecs = find_symp_mat_transvecs(X, Y)


# Print the results
# print("Resulting F matrix:")
print(F)
print("Applied Transvections:")
for mat, vec in Transvecs:
    print("Matrix:\n", mat, "\nVector:\n", vec)

a [[0. 0. 1. 0.]
 [0. 0. 1. 1.]]
b [1 1]
Ab [[0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 1.]]
Ab [[0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 1.]]
Ab [[0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 1.]]
Ab [[0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 1.]]


IndexError: index 3 is out of bounds for axis 0 with size 2